# Nebraska.Code
  
## Convert Celsius to Fahrenheit

Set environment to "Python 3.10 - SDK v2" to deploy the model

In [1]:
# Connect to Workspace

# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [2]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="5801c542-e7c2-4159-b473-1f8f689c28d0",
    resource_group_name="nurl_50-rg",
    workspace_name="nebraskacodeworkspacepayasyougo",
)

In [4]:
# Load Model

from azure.ai.ml.entities import Model

model = Model(
    path="CtoF_Model.h5", 
    name="ctofmodel", 
    description="Celsius to Fahrenheit Model"
)

In [5]:
# Register Model
registered_model = ml_client.models.create_or_update(model=model)

In [6]:
# Create Endpoint Name
import uuid

online_endpoint_name = "CelsiusToFahrenheit-" + str(uuid.uuid4())[:8]

In [7]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Convert Celsius to Fahrenheit",
    auth_mode="key",
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpoint celsiustofahrenheit-db550e9a provisioning state: Succeeded


In [8]:
# NOTE: Use if you already have an endpoint
## Retrieve Endpoint
#endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
#
#print(
#    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
#)

Endpoint "celsiustofahrenheit-e6075681" with provisioning state "Succeeded" is retrieved


In [8]:
# Create Environment
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [9]:
%%writefile {dependencies_dir}/conda.yaml
name: keras-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - pip=21.2.4
  - pip:
      - numpy==1.21.2
      - tensorflow==2.11.0
      - azureml-core==1.52.0
      - azureml-defaults==1.52.0

Overwriting ./dependencies/conda.yaml


In [10]:
from azure.ai.ml.entities import Environment

custom_env_name = "keras-env"

job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Celsius to Fahrenheit Conversion",
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
job_env = ml_client.environments.create_or_update(job_env)

print(
    f"Environment with name {job_env.name} is registered to workspace, the environment version is {job_env.version}"
)

Environment with name keras-env is registered to workspace, the environment version is 10


In [12]:
# NOTE: Use this if you already have an environment created
#job_env = ml_client.environments.get("keras-env", "10")

In [13]:
# Deploy the model
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

model = registered_model

# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="ctof-blue-deployment",
    endpoint_name=online_endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(code="./src", scoring_script="score.py"),
    instance_type="Standard_DS1_v2",
    instance_count=1,
    environment=job_env,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint CelsiusToFahrenheit-db550e9a exists


.........................................................................................................

In [15]:
# Test Deployed Model
# { "data": [0] }

result = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./request/Deg0.json",
    deployment_name="ctof-blue-deployment"
)
print( result )

In [17]:
# Test Deployed Model
# { "data": [0] }

result = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./request/Deg0.json",
    deployment_name="ctof-blue-deployment"
)
print( result )

[[31.999773025512695]]


In [19]:
result = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./request/Deg100.json",
    deployment_name="ctof-blue-deployment"
)
print( result )

[[211.99961853027344]]


In [ ]:
# Clean up resources
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)